In [23]:
#Helper functions to create table, insert and fetchbooks 
def createBookTable(conn):
    conn.execute('''CREATE TABLE IF NOT EXISTS BOOKS
         (ID INT PRIMARY KEY     NOT NULL,
         NAME           TEXT    NOT NULL,
         AUTHOR        TEXT     NOT NULL,
         TOPIC        CHAR(50),
         EDITION         INT);''')
    print ("Table created successfully")
    conn.close()
    
def insertBook(conn, book):
    sql = "INSERT into Books (id, name, author, topic, edition) values (?,?,?,?,?)"
    conn.execute(sql,(book.id, book.name,book.author, book.topic, book.edition))
    conn.commit()
    print("Book {} inserted".format(book.id))
    
def fetchBooks(conn, id=0):
    sql = "Select id, name, author, topic, edition from books"
    if id > 0:
        sql+=" where id = {}".format(id)
    conn.row_factory = sqlite3.Row 
    cursor = conn.execute(sql)
    books = []
    for row in cursor:
        books.append(Book(name=row['name'],id=row['id'],author=row['author'],topic=row['topic'],edition=row['edition']))   
    print ("Books fetched: " , books)
    return books

def updateBook(conn, book):
    sql = "update Books set edition = ? where id = ?"
    conn.execute(sql,(book.edition,book.id))
    conn.commit()
    print ("Book {} updated".format(book.id))
    
def deleteBook(conn, bookid):
    sql = "Delete from Books where id = ?"
    conn.execute(sql,(bookid,))
    conn.commit()
    print ("Book {} deleted".format(bookid))
    
def clearDB(conn):
    sql = "Delete from Books"
    conn.execute(sql)
    conn.commit()
    print ("All books deleted")


In [24]:
from flask import Flask, request, jsonify
from flask_marshmallow import Marshmallow
import os

app = Flask(__name__)
ma = Marshmallow(app)

import sqlite3
DB_NAME = "RESTTESTDB.db"

def getConnection():
    return sqlite3.connect(DB_NAME)

createBookTable(getConnection())
# Class BOOK
class Book:
    def __init__(self,**kwargs):
        self.id = kwargs.get('id')
        self.name = kwargs.get('name')
        self.author = kwargs.get('author')
        self.topic = kwargs.get('topic')
        self.edition = kwargs.get('edition')
    def __repr__(self):
        return self.name+' by '+self.author+ ' edition - ' + str(self.edition)

class BookSchema(ma.Schema):
    class Meta:
        # Fields to expose
        fields = ('id', 'name','author','topic','edition')


book_schema = BookSchema()
books_schema = BookSchema(many=True)


# endpoint to create new user
@app.route("/book", methods=["POST"])
def add_user():
    print("Entering insert")
    id = request.json['id']
    name = request.json['name']
    topic = request.json['topic']
    edition = request.json['edition']
    author = request.json['author']
    new_book = Book(id=id, name=name, topic=topic,edition=edition, author=author)
    conn = getConnection()
    insertBook(conn,new_book)
    conn.close()
    result = book_schema.jsonify(new_book)
    print("Finished insert")
    return result


# endpoint to get user detail by id
@app.route("/books", methods=["GET"])
def user_detail():
    print("Entering fetch")
    conn = getConnection()
    user = fetchBooks(conn)
    conn.close()
    result = books_schema.jsonify(user)
    print("Finished fetch")
    return result

# endpoint to get user detail by id
@app.route("/book/<id>", methods=["GET"])
def user_details(id):
    print("Entering fetchAll")
    conn = getConnection()
    user = fetchBooks(conn,int(id))
    if len(user) < 1:
        conn.close()
        return jsonify({"status":"404", "message":"Book not found"})
    conn.close()
    result = book_schema.jsonify(user[0])
    print("Finished fetchAll")
    return result


# endpoint to update user
@app.route("/book/<id>", methods=["PUT"])
def user_update(id):
    print("Entering update")
    conn = getConnection()
    book = fetchBooks(conn,int(id))
    if len(book) < 1:
        conn.close()
        return jsonify({"status":"404", "message":"Book not found"})
    book[0].edition = request.json['edition']
    updateBook(conn,book[0])
    conn.close()
    result= book_schema.jsonify(book[0])
    print("Finished update")
    return result

# endpoint to delete user
@app.route("/book/<id>", methods=["DELETE"])
def user_delete(id):
    print("Entering Delete")
    conn = getConnection()
    book = fetchBooks(conn,int(id))
    if len(book) < 1:
        conn.close()
        return jsonify({"status":"404", "message":"Book not found"})
    deleteBook(conn,id)
    conn.close()
    result = book_schema.jsonify(book[0]) 
    print("Finished Delete")
    return result

if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Table created successfully


In [1]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello World!'

if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Sep/2018 18:57:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2018 18:57:39] "GET / HTTP/1.1" 200 -
